In [23]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","RColorBrewer","zoo","lubridate","tidyverse",
              "readxl","gridExtra","MASS","ggpubr")
for(x in libraries) {library(x, character.only=TRUE, warn.conflicts=FALSE, quietly=TRUE)}

'%&%' = function(x,y)paste0(x,y)

theme_set(theme_bw())
version$version.string

options(scipen=10000)

[1] "R version 4.2.2 (2022-10-31)"

# Settings

In [24]:
readRDS("backproj_cases_non_Omi_MA.rds") %>% 
filter(onset >= as.Date("2020-01-15") & onset <= as.Date("2021-12-31")) -> temp
temp %>% mutate(t=0:(nrow(temp)-1)) -> backproj_Delta

readRDS("backproj_cases_Omi_MA.rds") %>% 
filter(onset >= as.Date("2020-01-15") & onset <= as.Date("2022-06-30")) -> temp
temp %>% mutate(t=0:(nrow(temp)-1)) -> backproj_Omicron

In [3]:
readRDS("RDS/immune_total_Delta_wDate.RDS") -> immune_total_Delta
readRDS("RDS/immune_total_Omi_wDate.RDS") -> immune_total_Omi

In [4]:
## implementation time of major NPIs
NPI_0 <- as.Date("2021-11-01") ## announcing the gradual ease of the social distancing countermeasures
NPI_1 <- as.Date("2021-12-06") ## state of emergency
NPI_2 <- as.Date("2022-02-19") ## extending the restricted operation hour(21 -> 22)
NPI_3 <- as.Date("2022-03-05") ## extending the restricted operation hour (22 -> 23)
NPI_4 <- as.Date("2022-04-04") ## extending the restricted operation hour (23 -> 24)
NPI_5 <- as.Date("2022-04-18") ## lifting all interventions

## Moving average
MA=7

## Generation time distribution

In [5]:
## generation interval (Nishiura, et al, 2020)
gi_fit = list(shape=2.305, scale=5.452)
generation <- function(t){pweibull(t, shape = gi_fit$shape, scale = gi_fit$scale) - 
                          pweibull(t-1, shape = gi_fit$shape, scale = gi_fit$scale)}

## generation time distribution (Abbott, 2022)
gi_fit_omi = list(shape=(3.3)^2/(3.5)^2, scale=(3.5)^2/3.3)
generation_omi <- function(t){pgamma(t, shape = gi_fit_omi$shape, scale = gi_fit_omi$scale) - 
                              pgamma(t-1, shape = gi_fit_omi$shape, scale = gi_fit_omi$scale)}

# Effective reproduction number in Delta period

In [ ]:
library(foreach); library(doParallel);
(ncore <- detectCores(logical = FALSE))

In [ ]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

backproj_Delta -> dt.backproj
Start.T <- 533  # Start from July 1, 2021

# estimating Re
est.t <- list(); est.CI <- list()

foreach(TT = Start.T:max(dt.backproj$t), .packages = c("dplyr")) %dopar% {
  dt.backproj %>% filter(t <= TT) -> dt.backproj.T
  
  llk <- function(param) {
    t <- TT
    R <- param
    Css <- sapply(1:(t-1), function(tau) dt.backproj.T$total[t - tau + 1] * generation(tau))
    Cs <- sum(Css) * R
    Cs[Cs <= 0] <- 1e-5
    return(-(-Cs + dt.backproj.T$total[t + 1] * log(Cs) - lgamma(dt.backproj.T$total[t + 1] + 1)))
  }

  opt_est <- optim(c(0.7), fn = llk, method = "L-BFGS-B", lower = 0, control = list(maxit = 10000))
  est.t[[TT]] <- opt_est$par

  # 95% confidence intervals using the profile likelihood
  par_CI <- seq(0, 25, by = 0.01)
  logLik <- sapply(par_CI, function(par) 2 * (-llk(par) + opt_est$value))
  data_CI <- data.frame(par_CI, logLik)
  data_CI$logLik[data_CI$logLik < (max(data_CI$logLik) - 3.84)] <- NA
  data_CI <- na.omit(data_CI)
  ci_pro <- data.frame(lower = min(data_CI$par_CI), upper = max(data_CI$par_CI))
  est.CI[[TT]] <- ci_pro
}

stopCluster(myCluster)

est_t <- do.call(rbind, est.t)
est_CI <- do.call(rbind, est.CI)

est <- cbind(est_t, est_CI)

In [22]:
est %>% as.data.frame() %>% mutate(t = Start.T:max(dt.backproj$t)) -> result
merge(dt.backproj, result, by='t', all.x=TRUE) -> result
colnames(result) <- c("t","onset","total","Rt","lower","upper")

In [ ]:
## 7-days moving average
result %>%
mutate(MA_Rt = zoo::rollmean(Rt, k=MA, align="right", fill=NA),
       MA_lower = zoo::rollmean(lower, k=MA, align="right", fill=NA),
       MA_upper = zoo::rollmean(upper, k=MA, align="right", fill=NA)) %>% na.omit() -> result_MA

saveRDS(result_MA, "result_non_Omi_smooth.rds")

# Effective reproduction number in Omicron period

In [ ]:
myCluster <- makeCluster(ncore-1, type = "PSOCK")
registerDoParallel(myCluster)

backproj_Omicron -> dt.backproj2
Start.T <- 700 ## start from Dec 15, 2021

# estimating Re
est.t <- list(); est.CI <- list()

foreach(TT = Start.T:max(dt.backproj2$t), .packages = c("dplyr")) %dopar% {
  dt.backproj2 %>% filter(t <= TT) -> dt.backproj.T
  
  llk <- function(param) {
    t <- TT
    R <- param
    Css <- sapply(1:(t-1), function(tau) dt.backproj.T$total[t - tau + 1] * generation_omi(tau))
    Cs <- sum(Css) * R
    Cs[Cs <= 0] <- 1e-5
    return(-(-Cs + dt.backproj.T$total[t + 1] * log(Cs) - lgamma(dt.backproj.T$total[t + 1] + 1)))
  }

  opt_est <- optim(c(0.7), fn = llk, method = "L-BFGS-B", lower = 0, control = list(maxit = 10000))
  est.t[[TT]] <- opt_est$par

  # 95% confidence intervals using the profile likelihood
  par_CI <- seq(0, 25, by = 0.01)
  logLik <- sapply(par_CI, function(par) 2 * (-llk(par) + opt_est$value))
  data_CI <- data.frame(par_CI, logLik)
  data_CI$logLik[data_CI$logLik < (max(data_CI$logLik) - 3.84)] <- NA
  data_CI <- na.omit(data_CI)
  ci_pro <- data.frame(lower = min(data_CI$par_CI), upper = max(data_CI$par_CI))
  est.CI[[TT]] <- ci_pro
}

stopCluster(myCluster)

est_t <- do.call(rbind, est.t)
est_CI <- do.call(rbind, est.CI)

est2 <- cbind(est_t, est_CI)

In [26]:
est2 %>% as.data.frame() %>% mutate(t = Start.T:max(dt.backproj2$t)) -> result2
merge(dt.backproj2, result2, by='t', all.x=TRUE) -> result2
colnames(result2) <- c("t","onset","total","Rt","lower","upper")

In [ ]:
## 7-days moving average
result2 %>%
mutate(MA_Rt = zoo::rollmean(Rt, k=MA, align="right", fill=NA),
       MA_lower = zoo::rollmean(lower, k=MA, align="right", fill=NA),
       MA_upper = zoo::rollmean(upper, k=MA, align="right", fill=NA)) %>% na.omit() -> result2_MA

saveRDS(result2_MA, "result_Omi_smooth.rds")

In [ ]:
readRDS("result_non_Omi_smooth.rds") -> result_non_Omi

adj1=0.356
scaling_parameter1=max(result_non_Omi$total)/max(result_non_Omi$upper[!is.na(result_non_Omi$upper)])

result_non_Omi %>% filter(date >= as.Date("2021-07-31") & date <= as.Date("2022-12-31")) %>%
    ggplot() + 
    geom_bar(aes(x=date, y=total), fill="#FAAB18" ,stat='identity', width=0.7) +
    geom_line(aes(x=date,y=MA_Rt*scaling_parameter1*adj1),color="#1380A1",size=0.7) +
    geom_ribbon(aes(ymax=MA_upper*scaling_parameter1*adj1, ymin=MA_lower*scaling_parameter1*adj1, x=date), 
                fill="#1380A1", alpha = 0.4) +
    ggtitle("Effective reproduction number in Delta period") +
    labs(x="\n Date of infection", y="Incidence \n") +
    labs(x="", y="") +
    theme(text = element_text(size=15, family="sans",color="black"),
          axis.text = element_text(size=15, family="sans",color="black"),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
          legend.title = element_text(size = 15),
          legend.text = element_text(size = 15),
          plot.title = element_text(size=18, family="sans",color="black")) +
    scale_x_date(date_breaks = "2 months", expand = c(0, 0),
                 labels = function(x) if_else(is.na(lag(x)) | !year(lag(x)) == year(x), 
                                              paste(month(x, label = TRUE), "\n", year(x)), 
                                              paste(month(x, label = TRUE)))) +
    scale_y_continuous(expand = c(0, 0),
                       sec.axis = sec_axis(~./(scaling_parameter1*adj1), breaks=c(0,1,2,3,4,5),  
                                           name = "")) +
    coord_cartesian(ylim = c(0,8000)) +
    geom_hline(yintercept=1*scaling_parameter1*adj1, linetype="dashed", color = "#1380A1", size =0.7) +
    geom_vline(xintercept=NPI_1, linetype="dashed", color="#990000", size=1) +
    geom_vline(xintercept=as.Date("2021-10-28"), linetype="dashed", color="#990000", size=1) -> fig_org_delta

                 
                 
readRDS("result_Omi_smooth.rds") -> result_Omi

adj2=0.45
scaling_parameter2=max(result_Omi$total)/max(result_Omi$upper[!is.na(result_Omi$upper)])

result_Omi %>% filter(date >= as.Date("2022-01-15") & date <= as.Date("2022-05-01")) %>%
    ggplot() + 
    geom_bar(aes(x=date, y=total), fill="#FAAB18" ,stat='identity', width=0.7) +
    geom_line(aes(x=date,y=MA_Rt*scaling_parameter2*adj2),color="#1380A1",size=0.7) +
    geom_ribbon(aes(ymax=MA_upper*scaling_parameter2*adj2, ymin=MA_lower*scaling_parameter2*adj2, x=date), 
                fill="#1380A1", alpha = 0.4) +
    ggtitle("Effective reproduction number in Omicron period") +
    labs(x="\n Date of infection", y="Incidence \n") +
    labs(x="", y="") +
    theme(text = element_text(size=15, family="sans",color="black"),
          axis.text = element_text(size=15, family="sans",color="black"),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
          legend.title = element_text(size = 15),
          legend.text = element_text(size = 15),
          plot.title = element_text(size=18, family="sans",color="black")) +
    scale_x_date(date_breaks = "1 months", expand = c(0, 0),
                 labels = function(x) if_else(is.na(lag(x)) | !year(lag(x)) == year(x), 
                                              paste(month(x, label = TRUE), "\n", year(x)), 
                                              paste(month(x, label = TRUE)))) +
    scale_y_continuous(expand = c(0, 0),
                       sec.axis = sec_axis(~./(scaling_parameter2*adj2), breaks=c(0,1,2,3,4,5), 
                                           name = "")) +
    coord_cartesian(ylim = c(0,550000)) +
    geom_hline(yintercept=1*scaling_parameter2*adj2, linetype="dashed", color = "#1380A1", size =0.7) +
    geom_vline(xintercept=NPI_2, linetype="dashed", color="#990000", size=1) +
    geom_vline(xintercept=NPI_3, linetype="dashed", color="#990000", size=1) +
    geom_vline(xintercept=NPI_4, linetype="dashed", color="#990000", size=1) +
    geom_vline(xintercept=NPI_5, linetype="dashed", color="#990000", size=1) -> fig_org_omicron                 
                 
                 
options(repr.plot.width=14,repr.plot.height=5)
ggarrange(fig_org_delta, ggplot() + theme_void(), fig_org_omicron, nrow = 1,widths = c(1, 0.05, 1), 
          labels = c("A", "", "B"), font.label = list(size = 30), vjust=1.2, hjust=0.5) -> Fig_Rt_combined

options(repr.plot.width=14,repr.plot.height=5)
annotate_figure(Fig_Rt_combined, 
                left=text_grob("Incident cases", size=16, rot=90),
                right=text_grob("Effective reproduction number", size=16, rot=270),
                bottom=text_grob("Date of infection", size=16))